# Example 1

In [1]:
# Let's load the package
library('pratto')

Loading required package: randomForest
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.


In [2]:
# Let's load the ML model
data('all_all_KNN') # The name of the loaded model is knn

In [3]:
# Let's load the testing set
data(testset)

In [4]:
# Let's see the structure of this data frame
str(testset)

'data.frame':	2034 obs. of  13 variables:
 $ cum_id : int  27 20 1 1 27 33 15 15 11 12 ...
 $ gal_id : int  81120 17141 9754 88707 80465 31606 5023 984 19082 86481 ...
 $ mas    : num  10.02 9.39 9.28 9.91 9.18 ...
 $ r      : num  0.419 1.776 2.403 0.532 0.323 ...
 $ v      : num  1.1 0.246 0.335 1.126 0.668 ...
 $ flag1  : Factor w/ 4 levels "X1","X2","X3",..: 2 2 3 1 1 2 3 3 3 4 ...
 $ flag2  : Factor w/ 2 levels "X0","X1": 2 1 1 2 2 1 1 1 1 2 ...
 $ fl1    : Factor w/ 2 levels "X0","X1": 1 1 1 2 2 1 1 1 1 1 ...
 $ fl2    : Factor w/ 2 levels "X0","X1": 2 2 1 1 1 2 1 1 1 1 ...
 $ fl3    : Factor w/ 2 levels "X0","X1": 1 1 2 1 1 1 2 2 2 1 ...
 $ fl4    : Factor w/ 2 levels "X0","X1": 1 1 1 1 1 1 1 1 1 2 ...
 $ fl5    : Factor w/ 2 levels "X0","X1": 2 2 2 1 1 2 2 2 2 1 ...
 $ flag1_t: Factor w/ 5 levels "NN","X1","X2",..: 5 3 4 5 2 2 4 3 2 1 ...


In [29]:
# Let's keep only with the 'r' and 'v' columns that will be used to predict, and
# save the real classification for future comparison.

cat        <- testset[, c(4,5)]
real_class <- testset$flag1

In [30]:
# Let's predict the proabability of being of each class using our ML
pred_prob <- get_class(cat, knn)

In [31]:
# Let's join all the information and separate the different classes with
#  a treshold of 0.5 in the probability

cat <- data.frame(cat, real_class, pred_prob)
pred_class <- vector()
pred_class[which(cat$X1 > 0.5)] = 'X1'
pred_class[which(cat$X2 > 0.5)] = 'X2'
pred_class[which(cat$X3 > 0.5)] = 'X3'
pred_class[which(cat$X4 > 0.5)] = 'X4'
cat <- data.frame(cat, pred_class)

In [32]:
str(cat)

'data.frame':	2034 obs. of  8 variables:
 $ r         : num  0.419 1.776 2.403 0.532 0.323 ...
 $ v         : num  1.1 0.246 0.335 1.126 0.668 ...
 $ real_class: Factor w/ 4 levels "X1","X2","X3",..: 2 2 3 1 1 2 3 3 3 4 ...
 $ X1        : num  0.365 0 0 0.286 0.46 ...
 $ X2        : num  0 0.4127 0.0159 0.0476 0.127 ...
 $ X3        : num  0.0952 0.5873 0.9841 0.1746 0.1111 ...
 $ X4        : num  0.54 0 0 0.492 0.302 ...
 $ pred_class: Factor w/ 4 levels "X1","X2","X3",..: 4 3 3 NA NA 1 3 3 NA NA ...


In [38]:
# Let's see the confusion matrix
confusionMatrix(cat$real_class, cat$pred_class)

Confusion Matrix and Statistics

          Reference
Prediction  X1  X2  X3  X4
        X1  54   2   2  38
        X2   9 220 134   5
        X3   4 125 682  18
        X4  26   2   5  69

Overall Statistics
                                          
               Accuracy : 0.7348          
                 95% CI : (0.7108, 0.7578)
    No Information Rate : 0.59            
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.5363          
                                          
 Mcnemar's Test P-Value : 0.01215         

Statistics by Class:

                     Class: X1 Class: X2 Class: X3 Class: X4
Sensitivity            0.58065    0.6304    0.8287   0.53077
Specificity            0.96774    0.8585    0.7430   0.97391
Pos Pred Value         0.56250    0.5978    0.8227   0.67647
Neg Pred Value         0.96998    0.8744    0.7509   0.95282
Prevalence             0.06667    0.2502    0.5900   0.09319
Detection Rate  